# Extract features via tsfresh python package

In [1]:
#baseDIR = '/home/pataki/synapse/gitParkinson/' # base directory of the github repo
#uncomment the line above (+ update) if you are running this notebook in an empty namespace
try: baseDIR
except NameError:
    print('Error: baseDIR not found!')
    
try: coreNum
except NameError:
    coreNum = 4 # default CPU cores

In [2]:
cd $baseDIR/sub2.1_tremor/

/home/pataki/synapse/gitParkinson/sub2.1_tremor


In [3]:
%run -i ../src/helperFuncs.py

In [4]:
%matplotlib inline

In [5]:
mainDF = mainDFCreator('tremorScore')

Train shape: (3667, 12)
Test shape:  (1500, 12)
Merged:      (5167, 12)


In [6]:
mainDF = mainDFtrimmer(fileMinLen=1, inputDF=mainDF, plot=False) # drop empty files

Remained shape: (5005, 14)


In [7]:
from tsfresh import extract_features

/home/pataki/.local/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [8]:
np.random.seed(42)
tsfreshDF = pd.DataFrame()
for ID in pb(mainDF.dataFileHandleId.tolist()):
    rawDF = dataByID(ID, mainDF)
    rawDF['id'] = ID
    extracted_features = extract_features(rawDF, column_id='id', column_sort='timestamp',
                                          n_jobs = coreNum, disable_progressbar = True)
    tsfreshDF = tsfreshDF.append(extracted_features)

100%|██████████| 5005/5005 [6:02:58<00:00,  3.43s/it]   


In [9]:
tsfreshDF.to_csv('featureDB/tsFresh_RAW.tsv', sep = '\t', index=False)

## Doing some filtering

In [10]:
tsfreshDF = tsfreshDF.fillna(tsfreshDF.mean())

In [11]:
nullCols = tsfreshDF.describe().T[tsfreshDF.describe().T['std'] == 0].index.tolist()

In [12]:
for i in nullCols: # drop columns containing only constant value
    tsfreshDF.pop(i);

In [13]:
tsfreshDF.reset_index().rename(columns = {'id':'dataFileHandleId'}).to_csv('featureDB/tsFresh_fillNA_dropConstant.tsv', sep = '\t', index=False)